In [1]:
import numpy as np

from os import listdir
from os.path import isdir
from PIL import Image
from matplotlib import pyplot
from numpy import savez_compressed
from numpy import load
from numpy import asarray
from numpy import load
from numpy import expand_dims

import matplotlib.pyplot as plt

from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder

from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Model
from keras.layers import Input

In [2]:
import cv2

In [3]:
from keras.models import load_model
HaarCascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'))
MyFaceNet = load_model('facenet_keras.h5')

In [4]:
def create_signature(directory):
    signatures = list()
    for filename in listdir(directory):
        #path = directory + filename
        
        gbr1 = cv2.imread(directory + filename)
    
        wajah = HaarCascade.detectMultiScale(gbr1,1.1,4)
    
        if len(wajah)>0:
            x1, y1, width, height = wajah[0]         
        else:
            x1, y1, width, height = 1, 1, 10, 10

        x1, y1 = abs(x1), abs(y1)
        x2, y2 = x1 + width, y1 + height

        gbr = cv2.cvtColor(gbr1, cv2.COLOR_BGR2RGB)
        gbr = Image.fromarray(gbr)                  # konversi dari OpenCV ke PIL
        gbr_array = asarray(gbr)

        face = gbr_array[y1:y2, x1:x2]                        

        face = Image.fromarray(face)                       
        face = face.resize((160,160))
        face = asarray(face)

        face = face.astype('float32')
        mean, std = face.mean(), face.std()
        face = (face - mean) / std

        face = expand_dims(face, axis=0)
        signature = MyFaceNet.predict(face)

        signatures.append(signature)
    return signatures

In [9]:
def load_dataset(directory):
    X, y = list(), list()
    
    for subdir in listdir(directory):
        path = directory + subdir + '/'
        
        if not isdir(path):                           # skip any files that might be in the dir
            continue
        
        signatures = create_signature(path)
        labels = [subdir for _ in range(len(signatures))]
        
        X.extend(signatures)
        y.extend(labels)
        
    return asarray(X), asarray(y)

In [10]:
trainX, trainy = load_dataset('training/')

In [11]:
trainX=np.squeeze(trainX, axis=1)

In [ ]:
trainXX.shape

In [12]:
encoder = LabelEncoder()
encoder.fit(trainy)
encoded_Y = encoder.transform(trainy)               # konversi label ke integer

y = np_utils.to_categorical(encoded_Y)         # konversi integer ke one_hot_matrix

In [ ]:
y.shape

In [13]:
model = keras.Sequential()

model.add(layers.Dense(20, input_dim=128, activation='relu'))
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(3, activation='softmax'))

In [14]:
opt = keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=['accuracy'])

In [15]:
model.fit(trainX, y, epochs=100, batch_size=150)

Epoch 1/100
1/1 [==============================] - 0s 4ms/step - loss: 1.1255 - accuracy: 0.3333
Epoch 2/100
1/1 [==============================] - 0s 2ms/step - loss: 1.1065 - accuracy: 0.3556
Epoch 3/100
1/1 [==============================] - 0s 2ms/step - loss: 1.0890 - accuracy: 0.4111
Epoch 4/100
1/1 [==============================] - 0s 2ms/step - loss: 1.0735 - accuracy: 0.4889
Epoch 5/100
1/1 [==============================] - 0s 2ms/step - loss: 1.0597 - accuracy: 0.5222
Epoch 6/100
1/1 [==============================] - 0s 997us/step - loss: 1.0467 - accuracy: 0.5333
Epoch 7/100
1/1 [==============================] - 0s 2ms/step - loss: 1.0365 - accuracy: 0.5333
Epoch 8/100
1/1 [==============================] - 0s 2ms/step - loss: 1.0280 - accuracy: 0.5333
Epoch 9/100
1/1 [==============================] - 0s 998us/step - loss: 1.0194 - accuracy: 0.5556
Epoch 10/100
1/1 [==============================] - 0s 2ms/step - loss: 1.0113 - accuracy: 0.5556
Epoch 11/100
1/1 [=======

1/1 [==============================] - 0s 2ms/step - loss: 0.1873 - accuracy: 0.9889
Epoch 84/100
1/1 [==============================] - 0s 2ms/step - loss: 0.1793 - accuracy: 0.9889
Epoch 85/100
1/1 [==============================] - 0s 998us/step - loss: 0.1716 - accuracy: 0.9889
Epoch 86/100
1/1 [==============================] - 0s 2ms/step - loss: 0.1643 - accuracy: 0.9889
Epoch 87/100
1/1 [==============================] - 0s 2ms/step - loss: 0.1573 - accuracy: 0.9889
Epoch 88/100
1/1 [==============================] - 0s 3ms/step - loss: 0.1507 - accuracy: 0.9889
Epoch 89/100
1/1 [==============================] - 0s 998us/step - loss: 0.1443 - accuracy: 0.9889
Epoch 90/100
1/1 [==============================] - 0s 2ms/step - loss: 0.1382 - accuracy: 0.9889
Epoch 91/100
1/1 [==============================] - ETA: 0s - loss: 0.1324 - accuracy: 0.98 - 0s 2ms/step - loss: 0.1324 - accuracy: 0.9889
Epoch 92/100
1/1 [==============================] - 0s 997us/step - loss: 0.1269 - ac

In [16]:
model.save('modelSmile.h5')

In [18]:
trainy

array(['normal', 'normal', 'normal', 'normal', 'normal', 'normal',
       'normal', 'normal', 'normal', 'normal', 'normal', 'normal',
       'normal', 'normal', 'normal', 'normal', 'normal', 'normal',
       'normal', 'normal', 'normal', 'normal', 'normal', 'normal',
       'normal', 'normal', 'normal', 'normal', 'normal', 'normal',
       'smile', 'smile', 'smile', 'smile', 'smile', 'smile', 'smile',
       'smile', 'smile', 'smile', 'smile', 'smile', 'smile', 'smile',
       'smile', 'smile', 'smile', 'smile', 'smile', 'smile', 'smile',
       'smile', 'smile', 'smile', 'smile', 'smile', 'smile', 'smile',
       'smile', 'smile', 'surprise', 'surprise', 'surprise', 'surprise',
       'surprise', 'surprise', 'surprise', 'surprise', 'surprise',
       'surprise', 'surprise', 'surprise', 'surprise', 'surprise',
       'surprise', 'surprise', 'surprise', 'surprise', 'surprise',
       'surprise', 'surprise', 'surprise', 'surprise', 'surprise',
       'surprise', 'surprise', 'surprise', '